In [70]:
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from argparse import Namespace
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm



In [71]:
# ## 词汇表与序列处理类定义

In [72]:
class Vocabulary:
    def __init__(self, token_to_idx=None):
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx
        self._idx_to_token = {idx: token for token, idx in self._token_to_idx.items()}

    def to_serializable(self):
        return {'token_to_idx': self._token_to_idx}

    @classmethod
    def from_serializable(cls, contents):
        return cls(**contents)

    def add_token(self, token):
        if token in self._token_to_idx:
            return self._token_to_idx[token]
        index = len(self._token_to_idx)
        self._token_to_idx[token] = index
        self._idx_to_token[index] = token
        return index

    def add_many(self, tokens):
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        return self._token_to_idx[token]

    def lookup_index(self, index):
        if index not in self._idx_to_token:
            raise KeyError(f"索引 {index} 不存在于词汇表中")
        return self._idx_to_token[index]

    def __str__(self):
        return f"<Vocabulary(size={len(self)})>"

    def __len__(self):
        return len(self._token_to_idx)


class SequenceVocabulary(Vocabulary):
    def __init__(self, token_to_idx=None, unk_token="<UNK>", mask_token="<MASK>",
                 begin_seq_token="<BEGIN>", end_seq_token="<END>"):
        super().__init__(token_to_idx)
        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

        self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)

    def to_serializable(self):
        contents = super().to_serializable()
        contents.update({
            'unk_token': self._unk_token,
            'mask_token': self._mask_token,
            'begin_seq_token': self._begin_seq_token,
            'end_seq_token': self._end_seq_token
        })
        return contents

    def lookup_token(self, token):
        return self._token_to_idx.get(token, self.unk_index)



In [73]:
# ## 数据向量化处理类

In [74]:
class SurnameVectorizer:
    def __init__(self, char_vocab, nationality_vocab):
        self.char_vocab = char_vocab
        self.nationality_vocab = nationality_vocab
        print(f"字符词汇表特殊标记：<BEGIN>:{char_vocab.begin_seq_index}, <END>:{char_vocab.end_seq_index}")

        # 测试样本向量化
        sample = "Zhang"
        vec, length = self.vectorize(sample)
        print(f"样本'{sample}'向量化结果:\n{vec}\n有效长度:{length}")

    def vectorize(self, surname, vector_length=-1):
        indices = [self.char_vocab.begin_seq_index]
        indices.extend(self.char_vocab.lookup_token(token) for token in surname)
        indices.append(self.char_vocab.end_seq_index)

        if vector_length < 0:
            vector_length = len(indices)

        vector = np.zeros(vector_length, dtype=np.int64)
        vector[:len(indices)] = indices
        vector[len(indices):] = self.char_vocab.mask_index
        return vector, len(indices)

    @classmethod
    def from_dataframe(cls, df):
        char_vocab = SequenceVocabulary()
        nationality_vocab = Vocabulary()

        for _, row in df.iterrows():
            for char in row.surname:
                char_vocab.add_token(char)
            nationality_vocab.add_token(row.nationality)

        return cls(char_vocab, nationality_vocab)

    @classmethod
    def from_serializable(cls, contents):
        char_vocab = SequenceVocabulary.from_serializable(contents['char_vocab'])
        nat_vocab = Vocabulary.from_serializable(contents['nationality_vocab'])
        return cls(char_vocab, nat_vocab)

    def to_serializable(self):
        return {
            'char_vocab': self.char_vocab.to_serializable(),
            'nationality_vocab': self.nationality_vocab.to_serializable()
        }



In [75]:
# ## 数据集类实现

In [76]:
class SurnameDataset(Dataset):
    def __init__(self, surname_df, vectorizer):
        self.surname_df = surname_df
        self._vectorizer = vectorizer

        # 检查 split 列存在性
        if 'split' not in self.surname_df.columns:
            raise ValueError("DataFrame 必须包含 'split' 列")

        # 初始化数据集划分
        self.train_df = self.surname_df[self.surname_df['split'] == 'train']
        self.val_df = self.surname_df[self.surname_df['split'] == 'val']
        self.test_df = self.surname_df[self.surname_df['split'] == 'test']

        # 计算最大序列长度
        self._max_seq_length = max(map(len, self.surname_df.surname)) + 2

        # 创建数据集划分字典
        self._lookup_dict = {
            'train': (self.train_df, len(self.train_df)),
            'val': (self.val_df, len(self.val_df)),
            'test': (self.test_df, len(self.test_df))
        }

        # 设置初始数据集划分
        self.set_split('train')

        # 计算类别权重 (修复后的正确代码)
        class_counts = self.train_df.nationality.value_counts().to_dict() 
        sorted_counts = sorted(
            class_counts.items(),
            key=lambda x: self._vectorizer.nationality_vocab.lookup_token(x[0])
        )
        frequencies = [count for _, count in sorted_counts]
        self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32)

    @classmethod
    def load_dataset_and_make_vectorizer(cls, csv_path):
        # 正确使用局部变量 df
        df = pd.read_csv(csv_path)
        train_df = df[df.split == 'train']
        return cls(df, SurnameVectorizer.from_dataframe(train_df)) 

    @classmethod
    def load_dataset_and_load_vectorizer(cls, csv_path, vectorizer_path):
        df = pd.read_csv(csv_path)
        with open(vectorizer_path) as fp:
            vectorizer = SurnameVectorizer.from_serializable(json.load(fp))
        return cls(df, vectorizer)

    def save_vectorizer(self, save_path):
        with open(save_path, 'w') as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        return self._vectorizer

    def set_split(self, split='train'):
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        row = self._target_df.iloc[index]
        surname_vector, vec_length = self._vectorizer.vectorize(row.surname, self._max_seq_length)
        nationality_index = self._vectorizer.nationality_vocab.lookup_token(row.nationality)
        return {
            'x_data': surname_vector,
            'y_target': nationality_index,
            'x_length': vec_length
        }

    def get_num_batches(self, batch_size):
        return len(self) // batch_size


def generate_batches(dataset, batch_size, shuffle=True, drop_last=True, device="cpu"):
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)
    for data_dict in dataloader:
        yield {key: tensor.to(device) for key, tensor in data_dict.items()}


def column_gather(y_out, x_lengths):
    x_lengths = x_lengths.long().cpu().numpy() - 1
    return torch.stack([y_out[i, length] for i, length in enumerate(x_lengths)])



In [77]:
# ## 模型架构定义

In [78]:
class ElmanRNN(nn.Module):
    def __init__(self, input_size, hidden_size, batch_first=False):
        super().__init__()
        self.rnn_cell = nn.RNNCell(input_size, hidden_size)
        self.batch_first = batch_first
        self.hidden_size = hidden_size

    def _initial_hidden(self, batch_size):
        return torch.zeros((batch_size, self.hidden_size))

    def forward(self, x_in, initial_hidden=None):
        if self.batch_first:
            batch_size, seq_size, feat_size = x_in.size()
            x_in = x_in.permute(1, 0, 2)
        else:
            seq_size, batch_size, feat_size = x_in.size()

        hidden = self._initial_hidden(batch_size).to(x_in.device) if initial_hidden is None else initial_hidden
        hiddens = []

        for t in range(seq_size):
            hidden = self.rnn_cell(x_in[t], hidden)
            hiddens.append(hidden)

        hiddens = torch.stack(hiddens)
        if self.batch_first:
            hiddens = hiddens.permute(1, 0, 2)
        return hiddens


class SurnameClassifier(nn.Module):
    def __init__(self, embedding_size, num_embeddings, num_classes, rnn_hidden_size,
                 batch_first=True, padding_idx=0):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_size, padding_idx=padding_idx)
        self.rnn = ElmanRNN(embedding_size, rnn_hidden_size, batch_first)
        self.fc1 = nn.Linear(rnn_hidden_size, rnn_hidden_size)
        self.fc2 = nn.Linear(rnn_hidden_size, num_classes)

    def forward(self, x_in, x_lengths=None, apply_softmax=False):
        x_embedded = self.emb(x_in)
        y_out = self.rnn(x_embedded)

        if x_lengths is not None:
            y_out = column_gather(y_out, x_lengths)
        else:
            y_out = y_out[:, -1, :]

        y_out = F.relu(self.fc1(F.dropout(y_out, 0.5)))
        y_out = self.fc2(F.dropout(y_out, 0.5))

        if apply_softmax:
            y_out = F.softmax(y_out, dim=1)

        # 调试输出
        print(f"输入形状: {x_in.shape} -> 嵌入后: {x_embedded.shape}")
        print(f"RNN输出形状: {y_out.shape if x_lengths is None else '动态索引'}")
        return y_out



In [79]:
# ## 初始化设置与参数配置

In [80]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)


def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)


args = Namespace(
    surname_csv="surnames_with_splits.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="model_storage",
    char_embedding_size=100,
    rnn_hidden_size=64,
    num_epochs=50,
    learning_rate=1e-3,
    batch_size=64,
    seed=1337,
    early_stopping_criteria=5,
    cuda=True,
    reload_from_files=False
)

# 硬件配置
if not torch.cuda.is_available():
    args.cuda = False
args.device = torch.device("cuda" if args.cuda else "cpu")
print(f"使用设备: {args.device}")

# 设置随机种子
set_seed_everywhere(args.seed, args.cuda)

# 创建保存目录
handle_dirs(args.save_dir)
args.vectorizer_file = os.path.join(args.save_dir, args.vectorizer_file)
args.model_state_file = os.path.join(args.save_dir, args.model_state_file)


使用设备: cpu


In [81]:
# ## 数据加载与预处理

In [82]:
if args.reload_from_files and os.path.exists(args.vectorizer_file):
    dataset = SurnameDataset.load_dataset_and_load_vectorizer(args.surname_csv, args.vectorizer_file)
else:
    dataset = SurnameDataset.load_dataset_and_make_vectorizer(args.surname_csv)
    dataset.save_vectorizer(args.vectorizer_file)

vectorizer = dataset.get_vectorizer()
print(f"\n数据集统计:")
print(f"训练样本数: {len(dataset.train_df)}")
print(f"验证样本数: {len(dataset.val_df)}")
print(f"测试样本数: {len(dataset.test_df)}")
print(f"国籍类别数: {len(vectorizer.nationality_vocab)}")


字符词汇表特殊标记：<BEGIN>:2, <END>:3
样本'Zhang'向量化结果:
[ 2 42  8  7 25 19  3]
有效长度:7

数据集统计:
训练样本数: 7680
验证样本数: 1640
测试样本数: 1660
国籍类别数: 18


In [83]:
# ## 模型初始化

In [84]:
classifier = SurnameClassifier(
    embedding_size=args.char_embedding_size,
    num_embeddings=len(vectorizer.char_vocab),
    num_classes=len(vectorizer.nationality_vocab),
    rnn_hidden_size=args.rnn_hidden_size,
    padding_idx=vectorizer.char_vocab.mask_index
).to(args.device)

print("\n模型结构:")
for name, param in classifier.named_parameters():
    print(f"{name:25} | 形状: {tuple(param.size())}")

# 损失函数与优化器
dataset.class_weights = dataset.class_weights.to(args.device)
loss_func = nn.CrossEntropyLoss(weight=dataset.class_weights)
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1)




模型结构:
emb.weight                | 形状: (80, 100)
rnn.rnn_cell.weight_ih    | 形状: (64, 100)
rnn.rnn_cell.weight_hh    | 形状: (64, 64)
rnn.rnn_cell.bias_ih      | 形状: (64,)
rnn.rnn_cell.bias_hh      | 形状: (64,)
fc1.weight                | 形状: (64, 64)
fc1.bias                  | 形状: (64,)
fc2.weight                | 形状: (18, 64)
fc2.bias                  | 形状: (18,)


In [85]:
# ## 训练辅助函数

In [86]:
def make_train_state(args):
    return {
        'stop_early': False,
        'early_stopping_step': 0,
        'early_stopping_best_val': float('inf'),
        'learning_rate': args.learning_rate,
        'epoch_index': 0,
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
        'test_loss': -1,
        'test_acc': -1,
        'model_filename': args.model_state_file
    }


def update_train_state(args, model, train_state):
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
    else:
        current_loss = train_state['val_loss'][-1]
        if current_loss < train_state['early_stopping_best_val']:
            torch.save(model.state_dict(), train_state['model_filename'])
            train_state['early_stopping_best_val'] = current_loss
            train_state['early_stopping_step'] = 0
        else:
            train_state['early_stopping_step'] += 1
            if train_state['early_stopping_step'] >= args.early_stopping_criteria:
                train_state['stop_early'] = True
    return train_state


def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    correct = torch.eq(y_pred_indices, y_target).sum().item()
    return correct / len(y_pred_indices) * 100



In [87]:
# ## 训练循环

In [88]:
# ## 最终测试评估

In [89]:
# ## 预测函数与示例

In [90]:
def predict_nationality(surname, classifier, vectorizer, device='cpu'):
    vec, length = vectorizer.vectorize(surname)
    vec_tensor = torch.tensor(vec).unsqueeze(0).to(device)
    length_tensor = torch.tensor([length]).to(device)

    with torch.no_grad():
        output = classifier(vec_tensor, length_tensor, apply_softmax=True)

    prob, index = torch.max(output, dim=1)
    return {
        'surname': surname,
        'nationality': vectorizer.nationality_vocab.lookup_index(index.item()),
        'probability': prob.item()
    }


test_samples = ["O'Neill", "Zhang", "Watanabe", "Schmidt"]
print("\n预测示例:")
for name in test_samples:
    prediction = predict_nationality(name, classifier, vectorizer, args.device)
    print(f"{prediction['surname']:10} → {prediction['nationality']:15} (概率: {prediction['probability']:.2%})")


预测示例:
输入形状: torch.Size([1, 9]) -> 嵌入后: torch.Size([1, 9, 100])
RNN输出形状: 动态索引
O'Neill    → German          (概率: 7.47%)
输入形状: torch.Size([1, 7]) -> 嵌入后: torch.Size([1, 7, 100])
RNN输出形状: 动态索引
Zhang      → Portuguese      (概率: 10.43%)
输入形状: torch.Size([1, 10]) -> 嵌入后: torch.Size([1, 10, 100])
RNN输出形状: 动态索引
Watanabe   → Korean          (概率: 8.82%)
输入形状: torch.Size([1, 9]) -> 嵌入后: torch.Size([1, 9, 100])
RNN输出形状: 动态索引
Schmidt    → Korean          (概率: 8.51%)


In [91]:
train_state = make_train_state(args)

try:
    for epoch in tqdm(range(args.num_epochs), desc="训练进度"):
        # 训练阶段
        classifier.train()
        dataset.set_split('train')
        train_loss, train_acc = 0.0, 0.0

        for batch in tqdm(generate_batches(dataset, args.batch_size, device=args.device),
                          desc=f"Epoch {epoch + 1} 训练", leave=False):
            optimizer.zero_grad()
            y_pred = classifier(batch['x_data'], batch['x_length'])
            loss = loss_func(y_pred, batch['y_target'])
            loss.backward()
            optimizer.step()

            train_loss += (loss.item() - train_loss) / (batch['x_data'].size(0))
            train_acc += (compute_accuracy(y_pred, batch['y_target']) - train_acc) / (batch['x_data'].size(0))

        # 验证阶段
        classifier.eval()
        dataset.set_split('val')
        val_loss, val_acc = 0.0, 0.0

        with torch.no_grad():
            for batch in tqdm(generate_batches(dataset, args.batch_size, device=args.device),
                              desc=f"Epoch {epoch + 1} 验证", leave=False):
                y_pred = classifier(batch['x_data'], batch['x_length'])
                loss = loss_func(y_pred, batch['y_target'])
                val_loss += (loss.item() - val_loss) / (batch['x_data'].size(0))
                val_acc += (compute_accuracy(y_pred, batch['y_target']) - val_acc) / (batch['x_data'].size(0))

        # 更新训练状态
        train_state['train_loss'].append(train_loss)
        train_state['train_acc'].append(train_acc)
        train_state['val_loss'].append(val_loss)
        train_state['val_acc'].append(val_acc)
        train_state = update_train_state(args, classifier, train_state)
        scheduler.step(val_loss)

        if train_state['stop_early']:
            print("提前终止训练")
            break

except KeyboardInterrupt:
    print("训练被用户中断")


训练进度:   0%|          | 0/50 [00:00<?, ?it/s]
Epoch 1 训练: 0it [00:00, ?it/s]

输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引



Epoch 1 训练: 3it [00:00, 22.56it/s]

输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引



Epoch 1 训练: 9it [00:00, 38.74it/s]

...

输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引



Epoch 50 验证: 21it [00:00, 60.92it/s]

输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引



训练进度: 100%|██████████| 50/50 [02:58<00:00,  3.57s/it]

输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引


In [92]:
classifier.load_state_dict(torch.load(train_state['model_filename']))
classifier.eval()

dataset.set_split('test')
test_loss, test_acc = 0.0, 0.0

with torch.no_grad():
    for batch in tqdm(generate_batches(dataset, args.batch_size, device=args.device),
                      desc="测试评估"):
        y_pred = classifier(batch['x_data'], batch['x_length'])
        loss = loss_func(y_pred, batch['y_target'])
        test_loss += (loss.item() - test_loss) / (batch['x_data'].size(0))
        test_acc += (compute_accuracy(y_pred, batch['y_target']) - test_acc) / (batch['x_data'].size(0))

train_state['test_loss'] = test_loss
train_state['test_acc'] = test_acc

print(f"\n测试结果:")
print(f"损失: {test_loss:.4f} | 准确率: {test_acc:.2f}%")



测试评估: 19it [00:00, 93.10it/s]

输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]

测试评估: 25it [00:00, 92.26it/s]

输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引
输入形状: torch.Size([64, 19]) -> 嵌入后: torch.Size([64, 19, 100])
RNN输出形状: 动态索引

测试结果:
损失: 0.6165 | 准确率: 13.59%
